In [1]:
import pandas as pd

import requests
import whois
import virustotal_python
from requests_html import HTMLSession
from collections import Counter
from urllib.parse import urlparse
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings

disable_warnings(InsecureRequestWarning)


options = Options()
options.headless = True
 
from collections import defaultdict
from urllib.request import urlopen, URLError
import socket


C:\Users\cvaal\AppData\Local\Temp\ipykernel_16352\3156865653.py:20: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


In [2]:
domain = 'reddit.com'

def get_pagerank(domain):
    headers = {'API-OPR':'4ok88wgckg8o0cgcswo4gc4kkc0sgocw0woww4o0'}
    url = 'https://openpagerank.com/api/v1.0/getPageRank?domains%5B0%5D=' + domain
    request = requests.get(url, headers=headers)
    if request.status_code == 200:
        result = request.json()
        if result['status_code'] == 200:
            pr_dec = result['response'][0]['page_rank_decimal']
            rank = result['response'][0]['rank']

            return pr_dec, rank

def whois_checker(domain):
    w = whois.whois(domain)
    domain_registrar = w.get('registrar')
    postal_code = w.get('registrant_postal_code')
    country = w.get('country')

    return  domain_registrar, postal_code, country

def virus_scan(domain):
    with virustotal_python.Virustotal("242305dedab935c6a39736d0749ae88f2b68f1e2f0c8e82c81d8955c5fd194d3") as vtotal:
        resp = vtotal.request(f"domains/{domain}")
        virus_data = resp.json()['data']['attributes']['last_analysis_stats']
        return virus_data['malicious']


def link_checker(domain):
    session = HTMLSession()
    
    r = session.get("https://"+domain, verify=False)
    unique_netlocs = Counter(urlparse(link).netloc for link in r.html.absolute_links)
    outbound_n = 0
    local_n = 0

    for link in unique_netlocs:
        # print(link, unique_netlocs[link])
        if domain.lower() in link:
            local_n += unique_netlocs[link]
        else:
            outbound_n += unique_netlocs[link]

    return local_n, local_n + outbound_n



def check_login(domain, driver):
    wp_xpath = "//a[starts-with(@href, 'https://wordpress.org')]"
    try:
        displayed = driver.find_element('xpath', wp_xpath).is_displayed()

        return displayed
    except:

        return False

def check_page_source(domain, driver):
    try:
        driver.get(f"https://{domain}/wp-admin")
        wp_pg = "wp-content" in driver.page_source
        return wp_pg
    except:

        return False

def check_wp(domain):

    service_obj = Service(executable_path="C:/Users/cvaal/chromedriver.exe")
    driver = webdriver.Chrome(options=options, service=service_obj)
    driver.implicitly_wait(1)
    driver.get(f"https://{domain}/wp-admin")
    login = check_login(domain, driver)
    pg_source = check_page_source(domain, driver)
    wp = None
    if login == False and pg_source == False:
        wp = False
    else:
        wp = True

    driver.close()
    return wp


def check_url( url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}
    response = requests.get('https://'+url, timeout=5, headers=headers)
   
    return response.status_code


def get_site_data(domain):
    # try:
    # print(check_url(domain))
    # if check_url(domain) == True:
    # try:
    pr_dec, rank = get_pagerank(domain)
    domain_registrar, postal_code, country = whois_checker(domain)
    malicious = virus_scan(domain)
    local_n, total_links= link_checker(domain)
    wp_check = check_wp(domain)
    return domain, pr_dec, rank, domain_registrar, postal_code, country, malicious, local_n, total_links, wp_check
# except:
    #     return('site down')
    # else:
    #     return 'Site down'
    # # except:
    # #     return 'Site down'
 

In [3]:
from urllib.parse import urlparse
urlparse('https://stackoverflow.com/questions/44113335/extract-domain-from-url-in-python').netloc.strip("www.")

'stackoverflow.com'

In [4]:
with open('FakeNews_URLs.txt', 'r') as f:
    fakenews_sites = f.read().split('\n')
    fakenews_sites = [site.strip() for site in fakenews_sites]


def get_and_organize_data(domain_list):
    columns = ['Domain', 'Page rank decimal', 'Site Rank', 'Domain registrar', 'Postal code', 'Country of origin', 'Malicious', 'No. of Local links','Total links','Wordpress?']
    site_df_dict = defaultdict(list)

    for domain in domain_list:
        if 'http' in domain:
            domain = urlparse(domain).netloc.strip("www.")
        print(domain)
        try:
            site_data = get_site_data(domain.lower())
            if type(site_data) == tuple:
                for column, data in zip(columns, site_data):
                    site_df_dict[column].append(data)                
            else:
                print('site down')
        except:
            print('site down')
            continue
            

    return pd.DataFrame(site_df_dict)



In [5]:
import joblib
ct = joblib.load('SitePreprocessor.joblib')
clf = joblib.load('XGBoostClassifier.joblib')

clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('standard_scaler',
                                                  StandardScaler(),
                                                  ['Page rank decimal',
                                                   'Site Rank',
                                                   'Total links']),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['Postal code',
                                                   'Country of origin']),
                                                 ('label_encoder',
                                                  OrdinalEncoder(),
                                                  ['Wordpress?',
                                                   'Malicious'])])),
                ('XGBoost',
                 XGBClassifier(base_score...
                               gamma=0.3, gpu_id=-1, importance_type=None,
                               interaction_constraints='', learning_rate=0.01,
                               max_delta_step=0, max_depth=3, min_child_weigh=3,
                               min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=6, num_parallel_tree=1, predictor='auto',
                               random_state=0, reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=1, subsample=1,
                               tree_method='exact', validate_parameters=1,
                               verbosity=None))])

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier



ct = joblib.load('SitePreprocessor.joblib')
clf = joblib.load('XGBoostClassifier.joblib')

def predict(domain_list):
    print('scanning sites:' , domain_list)
    data = get_and_organize_data(domain_list) #dataframe obj
    print(data)
    domains = data['Domain'].values
    features_df = data.drop('Domain', axis=1)
    
    
    preds = clf.predict(features_df)
    predictions = [round(value) for value in preds]

    return predictions

#idea: create instance of organizing object everytime the system is ran, collects all the links and returns a full dataframe

In [7]:
with open('FilipinoRealNews.txt', 'r') as f:
    fil_real_sites = f.read().split('\n')
    fil_real_sites= [site.strip() for site in fil_real_sites]


In [8]:
#predict(fil_real_sites[:5])

In [9]:
fil_fake_sites = ['https://balita.definitelyfilipino.com', 'https://pinoytrendingnews.net', 'https://www.thedailysentry.net', 'https://brigadanews.ph', 'https://bandera.inquirer.net']

In [10]:
predict(fil_fake_sites)

scanning sites: ['https://balita.definitelyfilipino.com', 'https://pinoytrendingnews.net', 'https://www.thedailysentry.net', 'https://brigadanews.ph', 'https://bandera.inquirer.net']
balita.definitelyfilipino.com
pinoytrendingnews.net
thedailysentry.net
brigadanews.ph
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
bandera.inquirer.net
                          Domain  Page rank decimal Site Rank   
0  balita.definitelyfilipino.com               2.83  34229397  \
1          pinoytrendingnews.net               3.15  15781332   
2             thedailysentry.net               4.10   3340028   
3                 brigadanews.ph               3.29  11566328   
4           bandera.inquirer.net               4.91    445655   

         Domain registrar Postal code Country of origin  Malicious   
0         Domain.com, LLC       32256                US          0  \
1           NAMECHEAP INC         101                IS          0   
2           NAMECHEAP IN

[0, 0, 0, 0, 0]

In [11]:
predict('reddit.com')

scanning sites: reddit.com
r
site down
e
site down
d
site down
d
site down
i
site down
t
site down
.
site down
c
site down
o
site down
m
site down
Empty DataFrame
Columns: []
Index: []


KeyError: 'Domain'

In [ ]:
for site in fil_real_sites:
    predict(site)

scanning:businessmirror.com.ph
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
('businessmirror.com.ph', 5.6, '16363', None, None, None, 0, 457, 465, True)
<class 'numpy.ndarray'>
businessmirror.com.ph : reliable site : 1
scanning:bworldonline.com
('bworldonline.com', 5.38, '29569', 'Network Solutions, LLC', None, None, 0, 151, 168, False)
<class 'numpy.ndarray'>
bworldonline.com : reliable site : 1
scanning:tribune.net.ph
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
('tribune.net.ph', 5.15, '78947', None, None, None, 0, 149, 157, True)
<class 'numpy.ndarray'>
tribune.net.ph : reliable site : 1
scanning:malaya.com.ph
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
('malaya.com.ph', 5.06, '124301', None, None, None, 0, 260, 265, True)
<class 'numpy.ndarray'>
malaya.com.ph : reliable site : 1
scanning:mb.com.ph
Error trying to connect to socket: closing socket - [Errno 11001]

Exception ignored in: <function Service.__del__ at 0x000001E7DA9C76D0>
Traceback (most recent call last):
  File "c:\Users\cvaal\anaconda3\envs\pt-tf-py3.10\lib\site-packages\selenium\webdriver\common\service.py", line 184, in __del__
    self.stop()
  File "c:\Users\cvaal\anaconda3\envs\pt-tf-py3.10\lib\site-packages\selenium\webdriver\common\service.py", line 145, in stop
    self.send_remote_shutdown_command()
  File "c:\Users\cvaal\anaconda3\envs\pt-tf-py3.10\lib\site-packages\selenium\webdriver\common\service.py", line 129, in send_remote_shutdown_command
    if not self.is_connectable():
  File "c:\Users\cvaal\anaconda3\envs\pt-tf-py3.10\lib\site-packages\selenium\webdriver\common\service.py", line 118, in is_connectable
    return utils.is_connectable(self.port)
  File "c:\Users\cvaal\anaconda3\envs\pt-tf-py3.10\lib\site-packages\selenium\webdriver\common\utils.py", line 102, in is_connectable
    socket_ = socket.create_connection((host, port), 1)
  File "c:\Users\cvaal\anacond

('manilastandard.net', 5.22, '58179', 'DOMAINPEOPLE, INC.', 'REDACTED FOR PRIVACY', 'PH', 0, 323, 334, True)
<class 'numpy.ndarray'>
manilastandard.net : reliable site : 1
scanning:manilatimes.net
('manilatimes.net', 5.74, '13046', 'Network Solutions, LLC', None, None, 0, 77, 96, False)
<class 'numpy.ndarray'>
manilatimes.net : reliable site : 1
scanning:journal.com.ph
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
('journal.com.ph', 5.07, '117292', None, None, None, 0, 125, 128, True)
<class 'numpy.ndarray'>
journal.com.ph : reliable site : 1
scanning:philstar.com
('philstar.com', 5.63, '15414', 'Network Solutions, LLC', None, None, 0, 196, 272, False)
<class 'numpy.ndarray'>
philstar.com : reliable site : 1
scanning:officialgazette.gov.ph
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
('officialgazette.gov.ph', 5.27, '46290', None, None, None, 0, 71, 85, True)
<class 'numpy.ndarray'>
officialgazette.gov.ph : 

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
predict('https://www.officialgazette.gov.ph/')

scanning:officialgazette.gov.ph
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
('officialgazette.gov.ph', 4.58, '62421', None, None, None, 0, 59, 72, False)
officialgazette.gov.ph : Unreliable site


In [ ]:
site_df_dict = defaultdict(list)

for rating, domains in site_dict.items():
    for i, domain in enumerate(domains):
        print(f'Analyzing Site {i+1}: {domain} of rating: {rating}')
        site_data = get_site_data(domain.lower(), rating)
        if type(site_data) == tuple:
            for column, data in zip(columns, site_data):
                if type(data) != list:
                    site_df_dict[column].append(data)
                else:
                    if any(site in data for site in fakenews_sites):
                        site_df_dict[column].append(True)
                    else:
                        site_df_dict[column].append(False)
            print('Data Gathered')
        else:
            print('site down')



NameError: name 'defaultdict' is not defined